<img src="./images/01-takeaways.png" width=700px>


In [1]:
import os
import json
import requests
from dotenv import load_dotenv
from pprint import pprint
from rich.console import Console

In [2]:
console = Console()

In [3]:
# Load environment variables from a file called .env
# There is a sample file .env.sample - rename this to .env and put your API keys there.
# https://console.groq.com/login has a free tier that uses the same interface as OpenAI
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:14]}...")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-it37ct...


In [4]:
MODEL = "gpt-4o-mini"
print(f"Model selected: {MODEL}")

Model selected: gpt-4o-mini


For demo, we will create our own OpenAI request class but in future we will use the OpenAI library.



Temperature - The OpenAI API incorporates a hyperparameter known as temperature that affects the computation of token probabilities when generating output through the large language model. The temperature value ranges from 0 to 2, with lower values indicating greater determinism and higher values indicating more randomness. Default is 1.

Low Temperature:

The bag is full of mostly blue marbles, with a few red and green. Low temperature means you're very likely to pull a blue marble, but you might occasionally get a red or green.

High Temperature:

The bag is filled with a mix of colors, and all colors are equally likely. High temperature means you're equally likely to pull any color, including the less common ones.

<img src="./images/temperature.png">

In [5]:
# We are essentially making a POST request to one endpoint and we create a convenience Class to pass in prompts, temperature and model.


class MyCustomOpenAI:

    def __init__(self, system_prompt=None, temperature=1.0, model=MODEL):
        self.model_endpoint = "https://api.openai.com/v1/chat/completions"
        self.temperature = temperature
        self.model = model
        self.system_prompt = system_prompt
        self.api_key = openai_api_key
        self.headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}",
        }

    # Note that other than the API key to be authenticated, we pass nor reference to previous queries. Each request is STATELESS and is independent of any other request.

    # That is why we append previous output to new queries so that the LLM has the full context or 'picture'.

    # OpenAI was trained with 'messages' in the form of a list of messages - SYSTEM, USER, ASSISTANT - so using this is most effective.

    # SYSTEM will contain the 'character' and instruction set of the Agent, USER will be the input to the Agent and ASSISTANT will be the output.

    # TAKEAWAY: WWe are appending the last output to the list of 'messages' so that the Agent has knowledge of the context - the history of the conversation.

    def generate_text(self, prompt):
        payload = {
            "model": self.model,
            "messages": [
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": prompt},
            ],
            "stream": False,  # no streaming or output
            "temperature": self.temperature,
        }
        # Use HTTP POST method from Requests library

        # In future examples we will use OpenAI library instead of requests directly but this is for demo purposes.
        response = requests.post(
            url=self.model_endpoint, headers=self.headers, data=json.dumps(payload)
        ).json()
        return response

In [6]:
# We create an instance of MyCustomOpenAI and pass in a request - just a convenience wrapper of making a REST API call.

# System Prompt sets up the character of the AI Agent and can be considered to be the API route we are creating and it it will be in Natural Language - explained later...

client = MyCustomOpenAI(
    model=MODEL,
    system_prompt="You give concise answers to questions with no more than 200 characters",
    temperature=0.0,  # as deterministic as possible
)

response = client.generate_text(
    "What is an the difference between Pydantic.ai and Lnagchain?"
)

In [7]:
# We can drill down into 'response' to get our choice of data
# The 'answer' is in the message with 'role': 'assistant'
console.print(response)
# print(response["choices"][0])
# print(response["choices"][0]["message"])

{
    'id': 'chatcmpl-BvQvubGnEVk42VJRSr0ifvWcBgAI9',
    'object': 'chat.completion',
    'created': 1753027926,
    'model': 'gpt-4o-mini-2024-07-18',
    'choices': [
        {
            'index': 0,
            'message': {
                'role': 'assistant',
                'content': 'Pydantic is a data validation and settings management library for Python, while 
Langchain is a framework for building applications with language models.',
                'refusal': None,
                'annotations': []
            },
            'logprobs': None,
            'finish_reason': 'stop'
        }
    ],
    'usage': {
        'prompt_tokens': 39,
        'completion_tokens': 27,
        'total_tokens': 66,
        'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
        'completion_tokens_details': {
            'reasoning_tokens': 0,
            'audio_tokens': 0,
            'accepted_prediction_tokens': 0,
            'rejected_prediction_tokens': 0
        }
    },
    'service_tier': 'default',
    'system_fingerprint': None
}

In [8]:
pprint(response["choices"][0]["message"]["content"])

('Pydantic is a data validation and settings management library for Python, '
 'while Langchain is a framework for building applications with language '
 'models.')
